In [1]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data

df=pd.read_csv('Annthyroid_02_v01.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att13,att14,att15,att16,att17,att18,att19,att20,att21,outlier
0,0.73,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.00060,0.015,0.120,0.082,0.146,0
1,0.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00025,0.030,0.143,0.133,0.108,0
2,0.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00190,0.024,0.102,0.131,0.078,0
3,0.64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00090,0.017,0.077,0.090,0.085,0
4,0.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00025,0.026,0.139,0.090,0.153,0


In [3]:
df.shape

(6802, 22)

In [4]:
# df to values
df = df.values

In [5]:
# Deep ExtraTrees
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Annthyroid.json")  
gc = GCForest(config)

In [17]:
# train test
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
# count class
len(np.unique(y_train))

2

In [8]:
  # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
  

[ 2018-04-25 08:13:08,905][cascade_classifier.fit_transform] X_groups_train.shape=[(4761, 21)],y_train.shape=(4761,),X_groups_test.shape=[(2041, 21)],y_test.shape=(2041,)
[ 2018-04-25 08:13:08,907][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-04-25 08:13:08,908][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 08:13:08,909][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-04-25 08:13:08,910][cascade_classifier.fit_transform] X_train.shape=(4761, 21),X_test.shape=(2041, 21)
[ 2018-04-25 08:13:08,912][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4761, 21), X_cur_test.shape=(2041, 21)
[ 2018-04-25 08:13:09,626][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.90%
[ 2018-04-25 08:13:10,433][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.16%
[ 2018-04-25 08:13:11,192][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtratTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 08:13:16,994][cascade_classifier.transform] X_groups_test.shape=[(2041, 21)]
[ 2018-04-25 08:13:16,996][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 08:13:16,998][cascade_classifier.transform] X_test.shape=(2041, 21)
[ 2018-04-25 08:13:17,000][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2041, 21)


Test Accuracy of ExtratTrees = 98.971093 %


In [10]:
#  predict_proba + Data 
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 08:13:18,547][cascade_classifier.transform] X_groups_test.shape=[(2041, 21)]
[ 2018-04-25 08:13:18,549][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 08:13:18,551][cascade_classifier.transform] X_test.shape=(2041, 21)
[ 2018-04-25 08:13:18,552][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2041, 21)


In [11]:
# New data
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [12]:
X_test_new.shape

(2041, 23)

In [13]:
X_train_new.shape

(4761, 23)

In [14]:
# Cascade ExtraTrees
config = load_json("./examples/Annthyroidgc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 08:13:20,089][cascade_classifier.fit_transform] X_groups_train.shape=[(4761, 23)],y_train.shape=(4761,),X_groups_test.shape=[(2041, 23)],y_test.shape=(2041,)
[ 2018-04-25 08:13:20,092][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 08:13:20,093][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 08:13:20,095][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 08:13:20,097][cascade_classifier.fit_transform] X_train.shape=(4761, 23),X_test.shape=(2041, 23)
[ 2018-04-25 08:13:20,099][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4761, 23), X_cur_test.shape=(2041, 23)
[ 2018-04-25 08:13:20,727][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.32%
[ 2018-04-25 08:13:21,544][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.79%
[ 2018-04-25 08:13:22,314][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

[ 2018-04-25 08:13:53,679][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=98.95%
[ 2018-04-25 08:13:54,472][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=98.74%
[ 2018-04-25 08:13:55,273][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=98.32%
[ 2018-04-25 08:13:56,034][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=99.37%
[ 2018-04-25 08:13:56,829][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.16%
[ 2018-04-25 08:13:57,636][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.79%
[ 2018-04-25 08:13:58,409][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.58%
[ 2018-04-25 08:13:59,322][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.37%


In [15]:
# deep
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100),i,'fois')

In [16]:
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/Annthyroidgc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-04-25 08:14:00,358][cascade_classifier.fit_transform] X_groups_train.shape=[(4761, 23)],y_train.shape=(4761,),X_groups_test.shape=[(2041, 23)],y_test.shape=(2041,)
[ 2018-04-25 08:14:00,360][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 08:14:00,361][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 08:14:00,363][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 08:14:00,364][cascade_classifier.fit_transform] X_train.shape=(4761, 23),X_test.shape=(2041, 23)
[ 2018-04-25 08:14:00,367][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4761, 23), X_cur_test.shape=(2041, 23)
[ 2018-04-25 08:14:01,015][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.53%
[ 2018-04-25 08:14:01,800][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.58%
[ 2018-04-25 08:14:02,584][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

[ 2018-04-25 08:14:33,817][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=99.16%
[ 2018-04-25 08:14:34,589][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=98.53%
[ 2018-04-25 08:14:35,364][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=97.90%
[ 2018-04-25 08:14:36,125][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=98.95%
[ 2018-04-25 08:14:36,955][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.16%
[ 2018-04-25 08:14:37,840][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.37%
[ 2018-04-25 08:14:38,624][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.58%
[ 2018-04-25 08:14:39,423][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.37%


('Test Accuracy of Extratrees = 99.314062 %', 0, 'fois')


[ 2018-04-25 08:14:44,146][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.53%
[ 2018-04-25 08:14:44,950][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.37%
[ 2018-04-25 08:14:45,765][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.32%
[ 2018-04-25 08:14:46,587][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.16%
[ 2018-04-25 08:14:47,350][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.37%
[ 2018-04-25 08:14:48,081][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.95%
[ 2018-04-25 08:14:49,051][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.16%
[ 2018-04-25 08:14:49,777][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.37%


[ 2018-04-25 08:15:20,589][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.58%
[ 2018-04-25 08:15:21,360][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.58%
[ 2018-04-25 08:15:22,191][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.16%
[ 2018-04-25 08:15:23,024][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=99.16%
[ 2018-04-25 08:15:23,177][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=99.08%
[ 2018-04-25 08:15:23,179][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=99.31%
[ 2018-04-25 08:15:23,181][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=99.08%
[ 2018-04-25 08:15:23,183][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=99.31%
[ 2018-04-25 08:15:23,185][

...

[ 2018-04-25 08:28:08,512][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.74%
[ 2018-04-25 08:28:09,327][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-25 08:28:10,105][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.32%
[ 2018-04-25 08:28:10,881][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.37%
[ 2018-04-25 08:28:11,685][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.95%
[ 2018-04-25 08:28:12,485][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.32%
[ 2018-04-25 08:28:13,260][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.58%
[ 2018-04-25 08:28:14,094][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.16%

[ 2018-04-25 08:28:41,563][cascade_classifier.fit_transform] X_train.shape=(4761, 23),X_test.shape=(2041, 23)
[ 2018-04-25 08:28:41,567][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4761, 23), X_cur_test.shape=(2041, 23)


('Test Accuracy of Extratrees = 99.265066 %', 18, 'fois')


[ 2018-04-25 08:28:42,181][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.74%
[ 2018-04-25 08:28:43,090][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.58%
[ 2018-04-25 08:28:43,907][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.32%
[ 2018-04-25 08:28:44,708][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.58%
[ 2018-04-25 08:28:45,476][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.74%
[ 2018-04-25 08:28:46,263][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.74%
[ 2018-04-25 08:28:47,066][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.37%
[ 2018-04-25 08:28:47,822][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.16%


('Test Accuracy of Extratrees = 99.167075 %', 19, 'fois')


# ('Test Accuracy of Extratrees = 99.461049 %', 15, 'fois')